# AMMI notes - Naive Bayes classifier for sentiment analysis
> Using a Naive Bayes classifier to classify reviews

- toc:true
- badges: true
- comments: true
- author: Gbetondji Dovonon
- categories: [notes,ammi,blog]

This is a simple implementation of Naive Bayes classifier for review rating. The classifier uses Baye's rule to determine the probability of a text belonging to a certain class, then we find the class with the highest probability. 

Let's not a token belonging to a sentence as $w_i$ where $i$ is its position in the sentence. We can find the probability $P(c | w_1 w_2 ... w_n)$ for a piece of text with $n$ tokens. Using Baye's rule, and assuming independence of the tokens with respect to each other, we have
$$
\begin{aligned}
P(c | w_1 w_2 ... w_n) & \propto P(c) P( w_1 w_2 ... w_n | c) \\
& \propto P(c) P( w_1 | c) P (w_2 | c) ... P( w_n | c) \\
& \propto P(c) \prod_{i=0}^{n} P( w_i | c) \\
& \propto \log P(c) + \sum_{i=0}^{n} \log P( w_i | c) 
\end{aligned}
$$

This model doesn't directly use the probability $P(c | w_1 w_2 ... w_n)$ but rather uses the likelihood $P( w_1 w_2 ... w_n | c)$ and prior $P(c)$. It therefore belongs to the family of generative models.

We can use counts to approximate those probabilties.

$$
P(w_i | c) = \frac{count(w_i)}{count(c)}
$$
with $count(w_i)$ being the number of times the token $w_i$ appears in class $c$ and $count(c)$ the number of examples in that class.
$$
P(c) = \frac{count(c)}{|\mathcal{D}|}
$$
with $count(c)$ the number of examples in that class and $|\mathcal{D}|$ the size of the dataset.

We will try the classifier on review data. We will combine labeled review data from Amazon, Imdb and Yelp.

In [ ]:
!wget -O amazon_cells_labelled.txt https://raw.githubusercontent.com/jean72human/nlp-2018-jean72human/master/project1/sentiment_labelled_sentences/amazon_cells_labelled.txt
!wget -O imdb_labelled.txt https://raw.githubusercontent.com/jean72human/nlp-2018-jean72human/master/project1/sentiment_labelled_sentences/imdb_labelled.txt
!wget -O yelp_labelled.txt https://raw.githubusercontent.com/jean72human/nlp-2018-jean72human/master/project1/sentiment_labelled_sentences/yelp_labelled.txt

We will use the nltk python package for preprocessing. More specifically, it will be the punkt tokenizer and wordnet lemmatizer.

In [ ]:
from nltk import word_tokenize
import nltk
import numpy as np
import re

nltk.download('punkt')
nltk.download('wordnet')

Now we have the code for the class. The notable methods here are: 
- `_train`: it collects the various statistics we need for our classifier
- `_predict`: uses those statisctics to make predictions
- `preprocess`: takes in the sentence and does the tokenization, normalization and lemmatization

I also added a method to load and save the model.

In [32]:
### NAIVE BAYES CLASS


class naive_classifier:
    def __init__(self, classes):
        """
        Initialization of the 
        """
        self.trained = False
        self.classes = classes
        self.nclasses = len(self.classes)
        
        self.likelihoods = {c : dict() for c in range(self.nclasses) }
        self.priors = [0 for i in range(self.nclasses)]
        self.vocabulary = []

    def _train(self, corpus):
        classCounts = [0 for i in range(self.nclasses)]
        ndoc = len(corpus)
        wordCounts = {c : dict() for c in range(self.nclasses)}
        for document in corpus:
            review = document[0]
            label = document[-1]
            classCounts[label] += 1
            for word in review:
                if word in wordCounts[label].keys():
                    wordCounts[label][word] += 1
                else:
                    wordCounts[label][word] = 1
                    
        for index in range(len(self.classes)):
            self.priors[index] = np.log(classCounts[index]/ndoc)
            self.vocabulary += list(wordCounts[index].keys())
        self.vocabulary = set(self.vocabulary)
        print ("Vocabulary size: ",len(self.vocabulary))
            
        for index in range(len(self.classes)):
            for word in self.vocabulary:
                if word in wordCounts[index]:
                    self.likelihoods[index][word] = np.log((wordCounts[index][word]+1)/(sum(wordCounts[index].values())+len(wordCounts[index])))
                else:
                    self.likelihoods[index][word] = np.log((1)/(sum(wordCounts[index].values())+len(wordCounts[index])))            
        
    def _read(self, document):
        toReturn = []
        with open(document) as f:
            for line in f.readlines():
                pair = line.split('\n')
                pair = pair[0].split('\t')
                review = re.sub(r"[,?!-()*&^%|'.,]","",pair[0]) ## normalization
                bag = self.preprocess(pair[0].lower())
                label = int(pair[1])
                toReturn.append((bag,label))
        return toReturn
    
    def preprocess(self, sentence):
        """
        preprocesses the sentences. Tokenizes them, then lemmatizes the token using the wordnet lemmatizer
        """
        import string
        from nltk.stem import WordNetLemmatizer
        wordnet_lemmatizer = WordNetLemmatizer()
        words = word_tokenize(sentence)
        toReturn = []
        for word in words:
            if (word not in string.punctuation):
                toReturn.append(wordnet_lemmatizer.lemmatize(word))
        return toReturn
    
    def train(self, documents, test=False, split_ratio=0.3):
        """
        Takes txt inputs and trains the classifier
        """
        corpus = []
        for doc in documents:
            print ("reading: ",doc)
            for review in self._read(doc):
                corpus.append(review)
                
        if test:
            np.random.shuffle(corpus)
            split_point = int(len(corpus) * split_ratio)
            test_data = corpus[:split_point]
            train_data = corpus[split_point:]
            self._train(train_data)
            test_acc = self._test(test_data)
            train_acc = self._test(train_data)
            print (len(train_data)," training items")
            print (len(test_data)," testing items")
            print ("Training done")
            print ("Train accuracy: ",train_acc)
            print ("Test accuracy: ",test_acc)
        else:
            self._train(corpus)
            print ("Training done")
        self.trained = True
        
    def _predict(self, sentence):
        """
        Takes tokenized input and outputs numerical class
        """
        import operator
        sumc = dict()
        for c in range(self.nclasses):
            sumc[c] = self.priors[c]
            for word in sentence:
                if word in self.vocabulary:
                    sumc[c] += self.likelihoods[c][word]
        return max(sumc.items(), key=operator.itemgetter(1))[0]
    
    def predict(self, text):
        """
        Tokenize sentence, predicts and output class
        """
        sentence = self.preprocess(text)
        return self._predict(sentence)
    
    def _test(self, data):
        n_items = len(data)
        n_correct = 0
        for document in data:
            review = document[0]
            label = document[-1]
            c = self._predict(review)
            if (c == label): n_correct += 1
        return n_correct / n_items  
    
    def export(self, name):
        import json
        
        toExport = {
            "likelihoods":self.likelihoods,
            "priors":self.priors,
            "vocabulary":self.vocabulary
        }
        
        np.save(name, toExport)
            
    def load(self, name):
        import json
        
        loaded = np.load(name)
            
        self.likelihoods = loaded.item().get("likelihoods")
        self.priors = loaded.item().get("priors")
        self.vocabulary = loaded.item().get("vocabulary")

We train and test the classifier. We can see that even a naive classifier can achieve a relatively good accuracy on simple problems.

In [37]:
classifier = naive_classifier(classes = ["positive", "negative"])
classifier.train(["./amazon_cells_labelled.txt",
                  "./imdb_labelled.txt",
                  "./yelp_labelled.txt"],
                 test=True,
                 split_ratio=0.2)

reading:  ./amazon_cells_labelled.txt
reading:  ./imdb_labelled.txt
reading:  ./yelp_labelled.txt
Vocabulary size:  4371
2400  training items
600  testing items
Training done
Train accuracy:  0.93875
Test accuracy:  0.83


We can now test the classifier with any data at all and see the result. Here is an example that belongs to the positive class.

In [38]:
test_text1 = "Mushoku Tensei is the greatest light novel series ever! It has great world building, foreshadowing and character development"

classifier.predict(test_text1)

1

Here is one that belongs to the negative class

In [29]:
test_text2 = "The new iPhone was not satisfactory. The innovation is not there anymore and prices are still through the roof"

classifier.predict(test_text2)

0